In [1]:
import pandas as pd 
import hopsworks 
from datetime import datetime, timedelta

import numpy as np



#Making the notebook able to fetch from the .env file
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
api_key = os.environ.get('hopsworks_api')
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()
mr = project.get_model_registry() 

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/564374
2024-10-14 19:51:18,062 WARNING: using legacy validation callback
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


In [8]:
start_date = datetime.now() - timedelta(hours=48)
print(start_date.strftime("%Y-%m-%d"))

2024-10-09


In [9]:
end_date = datetime.now() - timedelta(hours=24)
print(end_date.strftime("%Y-%m-%d"))

2024-10-10


In [10]:
feature_view = fs.get_feature_view('amd_stocks_fv', 8)
feature_view.init_batch_scoring(training_dataset_version=1)

In [11]:
df = feature_view.get_batch_data(start_time=start_date, end_time=end_date)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.74s) 


In [12]:
df.head()

,date,f_1__open,f_2__high,f_3__low,f_5__volume,ticker


In [14]:
import joblib

the_model = mr.get_model("ProphetModel", version=2)
model_dir = the_model.download()

model = joblib.load(model_dir + "/Prophet_model.pkl")

2024-10-14 20:00:01,326 WARNING: FutureIncompatibilityWarning: 

This is a future version incompatibility warning from Holidays library v0.58
to inform you about an upcoming change in our API versioning strategy that may affect your
project's dependencies. Starting from version 1.0 onwards, we will be following a loose form of
Semantic Versioning (SemVer, https://semver.org) to provide clearer communication regarding any
potential breaking changes.

This means that while we strive to maintain backward compatibility, there might be occasional
updates that introduce breaking changes to our API. To ensure the stability of your projects,
we highly recommend pinning the version of our API that you rely on. You can pin your current
holidays v0.x dependency (e.g., holidays==0.58) or limit it (e.g., holidays<1.0) in order to
avoid potentially unwanted upgrade to the version 1.0 when it's released (ETA 2024Q4 - 2025Q1).

If you have any questions or concerns regarding this change, please don't 

In [15]:
predictions = model.predict(df)

ValueError: Dataframe has no rows.

In [16]:
predictions 

2024-10-14 20:02:18,124 WARNING: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead

2024-10-14 20:02:18,125 WARNING: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead



NameError: name 'predictions' is not defined

In [ ]:
df['predictions'] = predictions.tolist()

In [ ]:
api_key = os.environ.get('hopsworks_api')
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()

In [ ]:
results_fg = fs.get_or_create_feature_group(
    name= 'stock_prediction_results',
    version = 1,
    description = 'Predction of AMD close stock price',
    primary_key = ['ticker'],
    event_time = ['date'],
    online_enabled = False,
)

In [ ]:
results_fg.insert(df, write_options={"wait_for_job" : False})